In [27]:
from z3 import *
import itertools

import time
import os

from sat_utils import plot_result, return_coods, optimize

os.makedirs('outputs', exist_ok=True)

In [28]:
def at_least_one(bool_vars):
    # TODO: implement at_least_one
    pass

def at_most_one(bool_vars):
    # TODO: at_most_one
    pass

In [45]:
def base_model_order(s,rects,px,py,under,left,W,H):
  # TODO:
  #    1) non overlapping condition 
  #    2) implicit domains
  #    3) order encoding 
  #    4) meaning for left and under:
  #         4.1) left(ri,rj) -> xj > wi, lower bound for xj
  #         4.2) for any e: left(ri,rj) -> (xi <e v ! xj<e+wi)
  #         4.3) under(ri,rj)-> yj > hi, lower bound for yj
  #         4.4) for any f, under(ri,rj) -> (yj <= f+hi ->yi <= f)
  pass
  return s

In [46]:
def solve(rects,W,opt=False):
  #TODO:
  #    1) set H_min and H_max
  #    2) loop condition  
  #    3) exit if out of time
  #    4) set variables px, py, under, left  
  #    5) solve problem (optimize) -->   
  #    6) update search horizon
  #    7) return solved_instance (bool), elapsed times, coordinates
  pass

In [ ]:
# TODO: plot results